## Process for PlanetScope Image Download

### Considerations:

1. The image was already downloaded from the PlanetScope repository and uploaded to Google Cloud Earth Engine Asset.
2. This was done to avoid the high time in calculating the indexes; it is more efficient to do this using geemap and/or Python.

### Index 

### 1. Prepare the image for download
#### 1.2 Case of Cafine
     1.2.1 The image exported to Drive needs to be downloaded using PyDrive
     1.2.2 Extract the data of the image using the sample points and applying a buffer over the points



### 3. Buffer for image statistics extraction and correlation with Sodium Absorption Rate (SAR)
   3.1. Call the image from GEE, download, and extract buffer data  
   3.2. Identify outliers to ensure optimal data conditions  
   3.3. Call the image and extract buffer data  
   3.4. Create a corrplot to visualize the correlation between different indices and SAR data  
   3.5. Export the index with the highest correlation for making predictions


 ## 1. Prepare the image for download 

### 1.2 Case of Cafine

In [7]:
# Import the libraries
import geemap
import ee
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import geopandas as gpd

In [2]:
# Authenticate your GEE account
ee.Authenticate() 


Successfully saved authorization token.


In [3]:
# Initialize GEE 
ee.Initialize()

In [4]:
# Add a map to display the image and index
Map = geemap.Map(center = (11.2114, -15.1915), zoom = 12)
Map

Map(center=[11.2114, -15.1915], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [5]:
# Ensure the creation of a geometry of the area for predict

geometry = ee.Geometry.Polygon([[[-15.21471, 11.188674],
   [-15.21471, 11.222857],
   [-15.133365, 11.222857],
   [-15.133365, 11.188674],
   [-15.21471, 11.188674]]])

In [22]:
# Call the image collection
collection = ee.ImageCollection("projects/planetscopejesus/assets/Datos_Gabo")

texture = ee.ImageCollection("projects/planetscopejesus/assets/Texture_cafine")
pca = ee.Image('projects/planetscopejesus/assets/pca_cafine').float()

# Filter by date (in this case is the same month of soil sampling)
collection = collection.filterDate('2022-05-01', '2022-05-30').filterBounds(geometry) # This is only to obtain the image (the image is from specific day and in this period of time we don't have more images)

# This function maps spectral indices for salinity Mapping using PlanetScope Imagery
def addIndices(img):
  # NDVI
  NDVI = img.normalizedDifference(['b8','b6']).rename('NDVI')
  SR = img.select('b8').divide(img.select('b6')).rename('SR')
  # GCVI
  GCVI = img.expression('(NIR/GREEN)-1',{'NIR':img.select('b8'),'GREEN':img.select('b3')}).rename('GCVI')
  # NDWI (GREEN-NIR)/(GREEN+NIR)
  NDWI = img.expression('(GREEN-NIR)/(GREEN+NIR)',{'NIR':img.select('b8'),'GREEN':img.select('b3')}).rename('NDWI')
  # VARI (Green−Red)/(Green+Reed−Blue)
  VARI = img.expression('(Green1-Red)/(Green1+Red-Blue)',{'Blue':img.select('b2'),'Red':img.select('b6'),'Green1':img.select('b3') }).rename('VARI')
  # GRVI (NIR/GREEN)
  GRVI = img.expression('(NIR/GREEN)',{'NIR':img.select('b8'),'GREEN':img.select('b3')}).rename('GRVI')
  # GNDVI (NIR-GREEN)/(NIR+GREEN)
  GNDVI = img.normalizedDifference(['b8','b3']).rename('GNDVI')
  # SAVI
  SAVI = img.expression('(NIR-RED)/((NIR+RED+0.5)*1.5)',{'NIR': img.select('b8'),'RED': img.select('b6')}).rename('SAVI')
  # VSSI = 2 ∗ 𝐵3 − 5(𝐵4 + 𝐵5) / Green(B3), Red(B4), NIR(B5)
  VSSI = img.expression('2*GREEN-5*(RED+NIR)',{'GREEN': img.select('b3'),'RED': img.select('b6'),'NIR': img.select('b8')}).rename('VSSI')
  # S1 (Blue/Red)
  S1 = img.select('b2').divide(img.select('b6')).rename('S1')
  # S1 (B − R)/(B + R)
  S2 = img.expression('(Blue-Red)/(Blue + Red)',{'Blue':img.select('b2'),'Red':img.select('b6') }).rename('S2')
  # S3_G1 (G1 × R)/B
  S3_G1 = img.expression('(Green1*Red)/(Blue)',{'Blue':img.select('b2'),'Red':img.select('b6'),'Green1':img.select('b3') }).rename('S3_G1')
  # S3_G2 (G1 × R)/B
  S3_G2 = img.expression('(Green2*Red)/(Blue)',{'Blue':img.select('b2'),'Red':img.select('b6'),'Green2':img.select('b4') }).rename('S3_G2')
  # S4 (B×R)**0.5
  S4 = img.expression('(Blue*Red)**0.5',{'Blue':img.select('b2'),'Red':img.select('b6')}).rename('S4')
  # S5_G1 ((B × R)/G1)
  S5_G1= img.expression('(Blue*Red)/Green1',{'Blue':img.select('b2'),'Green1':img.select('b3'),'Red':img.select('b6')}).rename('S5_G1')
  # S5_G2 ((B × R)/G1)
  S5_G2 = img.expression('(Blue*Red)/Green2',{'Blue':img.select('b2'),'Green2':img.select('b4'),'Red':img.select('b6')}).rename('S5_G2')
 # S6_G1 (R × NIR)/G1
  S6_G1 = img.expression('(Red*NIR)/Green1',{'NIR':img.select('b8'),'Green1':img.select('b3'),'Red':img.select('b6')}).rename('S6_G1')
  # S6_G2 (R × NIR)/G2
  S6_G2 = img.expression('(Red*NIR)/Green1',{'NIR':img.select('b8'),'Green1':img.select('b3'),'Red':img.select('b6')}).rename('S6_G2')
  # SI (B+R)**0.5
  SI = img.expression('(Blue+Red)**0.5',{'Blue':img.select('b2'),'Red':img.select('b6')}).rename('SI')
  # NDSI (R − NIR)/(R + NIR)
  NDSI = img.normalizedDifference(['b6','b8']).rename('NDSI')
  # SI1_G1 (G1×R)**0.5
  SI1_G1 = img.expression('(Green1*Red)**0.5',{'Green1':img.select('b3'),'Red':img.select('b6')}).rename('SI1_G1')
  # SI1_G2 (G1×R)**0.5
  SI1_G2 = img.expression('(Green2*Red)**0.5',{'Green2':img.select('b4'),'Red':img.select('b6')}).rename('SI1_G2')
  # SI2_G1 [(G1)**2 + (R)**2 + (NIR)**2]**0.5
  SI2_G1 = img.expression('((Green1)**2 + (Red)**2 + (NIR)**2)**0.5',{'NIR':img.select('b8'),'Green1':img.select('b3'),'Red':img.select('b6')}).rename('SI2_G1')
  # SI2_G2 [(G2)**2 + (R)**2 + (NIR)**2]**0.5
  SI2_G2 = img.expression('((Green2)**2 + (Red)**2 + (NIR)**2)**0.5',{'NIR':img.select('b8'),'Green2':img.select('b4'),'Red':img.select('b6')}).rename('SI2_G2')
  # SI3_G1 ((G1)**2 + (R)**2)**0.5
  SI3_G1 = img.expression('((Green1)**2 + (Red)**2)**0.5',{'Green1':img.select('b3'),'Red':img.select('b6')}).rename('SI3_G1')
  # SI3_G2 ((G1)**2 + (R)**2)**0.5
  SI3_G2 = img.expression('((Green2)**2 + (Red)**2)**0.5',{'Green2':img.select('b4'),'Red':img.select('b6')}).rename('SI3_G2')
  # Int1_G1 (G1 + R)/2
  Int1_G1 = img.expression('(Green1+Red)/2',{'Green1':img.select('b3'),'Red':img.select('b6')}).rename('Int1_G1')
  # Int1_G2 (G2 + R)/2
  Int1_G2 = img.expression('(Green2+Red)/2',{'Green2':img.select('b4'),'Red':img.select('b6')}).rename('Int1_G2')
  # Int2_G1 (G1 + R + NIR)/2
  Int2_G1 = img.expression('(Green1 + Red + NIR)/2',{'NIR':img.select('b8'),'Green1':img.select('b3'),'Red':img.select('b6')}).rename('Int2_G1')
  # Int2_G2 (G1 + R + NIR)/2
  Int2_G2 = img.expression('(Green2 + Red + NIR)/2',{'NIR':img.select('b8'),'Green2':img.select('b4'),'Red':img.select('b6')}).rename('Int2_G2')
  YRS6_G1 = img.expression('(Y*NIR)/G1',{'Y': img.select('b5'),'NIR': img.select('b8'),'G1': img.select('b3')}).rename('YRS6_G1')
  YRS6_G2 = img.expression('(Y*NIR)/G2',{'Y': img.select('b5'),'NIR': img.select('b8'), 'G2': img.select('b4')}).rename('YRS6_G2')
  RS6_G1 = img.expression('(Red_edge*NIR)/G1',{'Red_edge': img.select('b7'),'NIR': img.select('b8'),'G1': img.select('b3')}).rename('RS6_G1')
  RS6_G2 = img.expression('(Red_edge*NIR)/G2',{'Red_edge': img.select('b7'),'NIR': img.select('b8'),'G2': img.select('b4')}).rename('RS6_G2')
  RS1 = img.expression('B/Red_edge', {'B': img.select('b2'), 'Red_edge': img.select('b7')}).rename('RS1')
  RS2 = img.expression('(B - Red_edge) / (B + Red_edge)', {'B': img.select('b2'), 'Red_edge': img.select('b7')}).rename('RS2')
  RS3_G1 = img.expression('(G1 * Red_edge) / B', {'G1': img.select('b3'), 'Red_edge': img.select('b7'), 'B': img.select('b2')}).rename('RS3_G1')
  RS3_G2 = img.expression('(G2 * Red_edge) / B', {'G2': img.select('b4'), 'Red_edge': img.select('b7'), 'B': img.select('b2')}).rename('RS3_G2')
  RS4 = img.expression('(B * Red_edge) ** 0.5', {'B': img.select('b2'), 'Red_edge': img.select('b7')}).rename('RS4')
  RS5_G1 = img.expression('(B * Red_edge) / G1', {'B': img.select('b2'), 'Red_edge': img.select('b7'), 'G1': img.select('b3')}).rename('RS5_G1')
  RS5_G2 = img.expression('(B * Red_edge) / G2', {'B': img.select('b2'), 'Red_edge': img.select('b7'), 'G2': img.select('b4')}).rename('RS5_G2')
  RS6_G1 = img.expression('(Red_edge * NIR) / G1', {'Red_edge': img.select('b7'), 'NIR': img.select('b8'), 'G1': img.select('b3')}).rename('RS6_G1')
  RS6_G2 = img.expression('(Red_edge * NIR) / G2', {'Red_edge': img.select('b7'), 'NIR': img.select('b8'), 'G2': img.select('b4')}).rename('RS6_G2')
  RNDSI = img.expression('(Red_edge - NIR) / (Red_edge + NIR)', {'Red_edge': img.select('b7'), 'NIR': img.select('b8')}).rename('RNDSI')
  RNDVI = img.expression('(NIR - Red_edge) / (NIR + Red_edge)', {'Red_edge': img.select('b7'), 'NIR': img.select('b8')}).rename('RNDVI')
  RSI = img.expression('(B + Red_edge) ** 0.5', {'B': img.select('b2'), 'Red_edge': img.select('b7')}).rename('RSI')
  RSI1_G1 = img.expression('(G1 * Red_edge) ** 0.5', {'G1': img.select('b3'), 'Red_edge': img.select('b7')}).rename('RSI1_G1')
  RSI1_G2 = img.expression('(G2 * Red_edge) ** 0.5', {'G2': img.select('b4'), 'Red_edge': img.select('b7')}).rename('RSI1_G2')
  RSI2_G1 = img.expression('((G1) ** 2 + (Red_edge) ** 2 + (NIR) ** 2) ** 0.5', {'G1': img.select('b3'), 'Red_edge': img.select('b7'), 'NIR': img.select('b8')}).rename('RSI2_G1')
  RSI2_G2 = img.expression('((G2) ** 2 + (Red_edge) ** 2 + (NIR) ** 2) ** 0.5', {'G2': img.select('b4'), 'Red_edge': img.select('b7'), 'NIR': img.select('b8')}).rename('RSI2_G2')
  RSI3_G1 = img.expression('((G1) ** 2 + (Red_edge) ** 2) ** 0.5', {'G1': img.select('b3'), 'Red_edge': img.select('b7')}).rename('RSI3_G1')
  RSI3_G2 = img.expression('((G2) ** 2 + (Red_edge) ** 2) ** 0.5', {'G2': img.select('b4'), 'Red_edge': img.select('b7')}).rename('RSI3_G2')
  RInt1_G1 = img.expression('(G1 + Red_edge) / 2', {'G1': img.select('b3'), 'Red_edge': img.select('b7')}).rename('RInt1_G1')
  RInt1_G2 = img.expression('(G2 + Red_edge) / 2', {'G2': img.select('b4'), 'Red_edge': img.select('b7')}).rename('RInt1_G2')
  RInt2_G1 = img.expression('(G1 + Red_edge + NIR) / 2', {'G1': img.select('b3'), 'Red_edge': img.select('b7'), 'NIR': img.select('b8')}).rename('RInt2_G1')
  RInt2_G2 = img.expression('(G2 + Red_edge + NIR) / 2', {'G2': img.select('b4'), 'Red_edge': img.select('b7'), 'NIR': img.select('b8')}).rename('RInt2_G2')
  ####################################################################################### Table 4 ######################################################################################
  YRS1 = img.expression('B/Y', {'B': img.select('b2'), 'Y': img.select('b5')}).rename('YRS1')
  YRS2 = img.expression('(B - Y) / (B + Y)', {'B': img.select('b2'), 'Y': img.select('b5')}).rename('YRS2')
  YRS3_G1 = img.expression('(G1 * Y) / B', {'G1': img.select('b3'), 'Y': img.select('b5'), 'B': img.select('b2')}).rename('YRS3_G1')
  YRS3_G2 = img.expression('(G2 * Y) / B', {'G2': img.select('b4'), 'Y': img.select('b5'), 'B': img.select('b2')}).rename('YRS3_G2')
  YRS4 = img.expression('(B * Y) ** 0.5', {'B': img.select('b2'), 'Y': img.select('b5')}).rename('YRS4')
  YRS5_G1 = img.expression('(B * Y) / G1', {'B': img.select('b2'), 'Y': img.select('b5'), 'G1': img.select('b3')}).rename('YRS5_G1')
  YRS5_G2 = img.expression('(B * Y) / G2', {'B': img.select('b2'), 'Y': img.select('b5'), 'G2': img.select('b4')}).rename('YRS5_G2')
  YRS6_G1 = img.expression('(Y * NIR) / G1', {'Y': img.select('b5'), 'NIR': img.select('b8'), 'G1': img.select('b3')}).rename('YRS6_G1')
  YRS6_G2 = img.expression('(Y * NIR) / G2', {'Y': img.select('b5'), 'NIR': img.select('b8'), 'G2': img.select('b4')}).rename('YRS6_G2')
  YRSI = img.expression('(B + Y) ** 0.5', {'B': img.select('b2'), 'Y': img.select('b5')}).rename('YRSI')
  YRSI1_G1 = img.expression('(G1 * Y) ** 0.5', {'G1': img.select('b3'), 'Y': img.select('b5')}).rename('YRSI1_G1')
  YRSI1_G2 = img.expression('(G2 * Y) ** 0.5', {'G2': img.select('b4'), 'Y': img.select('b5')}).rename('YRSI1_G2')
  YRSI2_G1 = img.expression('(((G1) ** 2) + ((Y) ** 2) + ((NIR) ** 2)) ** 0.5', {'G1': img.select('b3'), 'Y': img.select('b5'), 'NIR': img.select('b8')}).rename('YRSI2_G1')
  YRSI2_G2 = img.expression('(((G2) ** 2) + ((Y) ** 2) + ((NIR) ** 2)) ** 0.5', {'G2': img.select('b4'), 'Y': img.select('b5'), 'NIR': img.select('b8')}).rename('YRSI2_G2')
  YRSI3_G1 = img.expression('(((G1) ** 2) + ((Y) ** 2)) ** 0.5', {'G1': img.select('b3'), 'Y': img.select('b5')}).rename('YRSI3_G1')
  YRSI3_G2 = img.expression('(((G2) ** 2) + ((Y) ** 2)) ** 0.5', {'G2': img.select('b4'), 'Y': img.select('b5')}).rename('YRSI3_G2')
  YRInt1_G1 = img.expression('(G1 + Y) / 2', {'G1': img.select('b3'), 'Y': img.select('b5')}).rename('YRInt1_G1')
  YRInt1_G2 = img.expression('(G2 + Y) / 2', {'G2': img.select('b4'), 'Y': img.select('b5')}).rename('YRInt1_G2')
  YRInt2_G1 = img.expression('(G1 + Y + NIR) / 2', {'G1': img.select('b3'), 'Y': img.select('b5'), 'NIR': img.select('b8')}).rename('YRInt2_G1')
  YRInt2_G2 = img.expression('(G2 + Y + NIR) / 2', {'G2': img.select('b4'), 'Y': img.select('b5'), 'NIR': img.select('b8')}).rename('YRInt2_G2')
  YRNDSI = img.expression('(Y - NIR) / (Y + NIR)', {'Y': img.select('b5'), 'NIR': img.select('b8')}).rename('YRNDSI')
  ##########################################################
  YRNDVI = img.expression('(NIR - Y) / (NIR + Y)', {'NIR': img.select('b8'), 'Y': img.select('b5')}).rename('YRNDVI')
  YBS1 = img.expression('Y / R', {'Y': img.select('b5'), 'R': img.select('b6')}).rename('YBS1')
  YBS2 = img.expression('(Y - R) / (Y + R)', {'Y': img.select('b5'), 'R': img.select('b6')}).rename('YBS2')
  YBS4 = img.expression('(Y * R) ** 0.5', {'Y': img.select('b5'), 'R': img.select('b6')}).rename('YBS4')
  YBS5_G1 = img.expression('(Y * R) / G1', {'Y': img.select('b5'), 'R': img.select('b6'), 'G1': img.select('b3')}).rename('YBS5_G1')
  YBS5_G2 = img.expression('(Y * R) / G2', {'Y': img.select('b5'), 'R': img.select('b6'), 'G2': img.select('b4')}).rename('YBS5_G2')
  YBSI = img.expression('(Y + R) ** 0.5', {'Y': img.select('b5'), 'R': img.select('b6')}).rename('YBSI')
  YGS3 = img.expression('(Y * R) / B', {'B': img.select('b2'), 'Y': img.select('b5'), 'R': img.select('b6')}).rename('YGS3')
  YGSI1 = img.expression('(Y * R) ** 0.5', {'R': img.select('b6'), 'Y': img.select('b5')}).rename('YGSI1')
  YGSI2 = img.expression('((Y ** 2) + (R ** 2) + (NIR ** 2)) ** 0.5', {'R': img.select('b6'), 'Y': img.select('b5'), 'NIR': img.select('b8')}).rename('YGSI2')
  YGSI3 = img.expression('((Y ** 2) + (R ** 2)) ** 0.5', {'Y': img.select('b5'), 'R': img.select('b6')}).rename('YGSI3')
  YGInt1 = img.expression('(Y + R) / 2', {'R': img.select('b6'), 'Y': img.select('b5')}).rename('YGInt1')
  YGInt2 = img.expression('(Y + R + NIR) / 2', {'R': img.select('b6'), 'Y': img.select('b5'), 'NIR': img.select('b8')}).rename('YGInt2')
  YNS6_G1 = img.expression('(R * Y) / G1', {'R': img.select('b6'), 'Y': img.select('b5'), 'G1': img.select('b3')}).rename('YNS6_G1')
  YNS6_G2 = img.expression('(R * Y) / G2', {'R': img.select('b6'), 'Y': img.select('b5'), 'G2': img.select('b4')}).rename('YNS6_G2')
  YNSI2_G1 = img.expression('((G1 ** 2) + (R ** 2) + (Y ** 2)) ** 0.5', {'G1': img.select('b3'), 'Y': img.select('b5'), 'R': img.select('b6')}).rename('YNSI2_G1')
  YNSI2_G2 = img.expression('((G2 ** 2) + (R ** 2) + (Y ** 2)) ** 0.5', {'G2': img.select('b4'), 'Y': img.select('b5'), 'R': img.select('b6')}).rename('YNSI2_G2')
  YNInt2_G1 = img.expression('(G1 + R + Y) / 2', {'R': img.select('b6'), 'Y': img.select('b5'), 'G1': img.select('b3')}).rename('YNInt2_G1')
  YNInt2_G2 = img.expression('(G2 + R + Y) / 2', {'R': img.select('b6'), 'Y': img.select('b5'), 'G2': img.select('b4')}).rename('YNInt2_G2')
  YNNDSI = img.expression('(R - Y) / (R + Y)', {'Y': img.select('b5'), 'R': img.select('b6')}).rename('YNNDSI')
  YNNDVI = img.expression('(Y - R) / (Y + R)', {'R': img.select('b6'), 'Y': img.select('b5')}).rename('YNNDVI')
  return img \
    .addBands(
[
    NDVI, SR, GCVI, NDWI, VARI, GNDVI, GRVI, SAVI, VSSI, S1, S2, S3_G1, S3_G2, S4, S5_G1, S5_G2,
    S6_G1, S6_G2, SI, NDSI, SI1_G1, SI1_G2, SI2_G1, SI2_G2, SI3_G1, SI3_G2, Int1_G1, Int1_G2,
    Int2_G1, Int2_G2, YRS6_G1, YRS6_G2, RS6_G1, RS6_G2, RS1, RS2, RS3_G1, RS3_G2, RS4, RS5_G1,
    RS5_G2, RNDSI, RNDVI, RSI, RSI1_G1, RSI1_G2, RSI2_G1, RSI2_G2, RSI3_G1, RSI3_G2, RInt1_G1,
    RInt1_G2, RInt2_G1, RInt2_G2,YRS1, YRS2, YRS3_G1, YRS3_G2, YRS4, YRS5_G1, YRS5_G2, YRSI, YRSI1_G1, 
    YRSI1_G2, YRSI2_G1, YRSI2_G2, YRSI3_G1, YRSI3_G2, YRInt1_G1, YRInt1_G2, YRInt2_G1, YRInt2_G2, YRNDSI,
    YRNDVI, YBS1, YBS2, YBS4, YBS5_G1, YBS5_G2, YBSI, YGS3, YGSI1, YGSI2, YGSI3, YGInt1,
     YGInt2, YNS6_G1, YNS6_G2, YNSI2_G1, YNSI2_G2, YNInt2_G1, YNInt2_G2, YNNDSI, YNNDVI

]

    )

ps = collection

#Add the indices
ps_1 = ps.map(addIndices)
composite = ps_1 \
              .mean()\
              .float()

texture1 = texture.mean()


# Select the NIR band, for the calculation type it must be in Int32, since glcmTexture only supports this data type.
nir = composite.select('b8').toInt32()

# Calculate the texture using the Contrast Index
texture = nir.glcmTexture()
texture_b8 = texture.select('b8_asm').float()

# Add the all bands in only one file
composite = composite.addBands(texture1).addBands(pca).addBands(texture_b8)

# Add texture layer to the map and other images (index)
Map.addLayer(texture, {}, 'Textura NIR')
Map.addLayer(composite, {'bands': ['b6',  'b4',  'b2'], 'min': 201, 'max': 2464}, 'RGB')
Map.addLayer(composite, {'bands': ['S1',  'S2',  'S3_G1'], 'min': 0, 'max': 1}, 'Index')


# RELEVANT NOTE on soil texture raster order Cafine case:
# B1_1 = Clay + Silt
# B2_1 = Sand
# B3_1 = Silt
# B4_1 = Clay

#Export the bands to model Cafine.
export = composite.select(['b1','b2','b3','b4','b5','b6','b7','b8',"NDVI", "SR", "GCVI", "NDWI", "VARI", "GNDVI", "GRVI", "SAVI", "VSSI", "S1", "S2", 
"S3_G1", "S3_G2", "S4", "S5_G1", "S5_G2", "S6_G1", "S6_G2", "SI", "NDSI", "SI1_G1", 
"SI1_G2", "SI2_G1", "SI2_G2", "SI3_G1", "SI3_G2", "Int1_G1", "Int1_G2", "Int2_G1", 
"Int2_G2", "YRS6_G1", "YRS6_G2", "RS6_G1", "RS6_G2", "RS1", "RS2", "RS3_G1", "RS3_G2", 
"RS4", "RS5_G1", "RS5_G2", "RNDSI", "RNDVI", "RSI", "RSI1_G1", "RSI1_G2", "RSI2_G1", 
"RSI2_G2", "RSI3_G1", "RSI3_G2", "RInt1_G1", "RInt1_G2", "RInt2_G1", "RInt2_G2", 
"YRS1", "YRS2", "YRS3_G1", "YRS3_G2", "YRS4", "YRS5_G1", "YRS5_G2", "YRSI", "YRSI1_G1", 
"YRSI1_G2", "YRSI2_G1", "YRSI2_G2", "YRSI3_G1", "YRSI3_G2", "YRInt1_G1", "YRInt1_G2", 
"YRInt2_G1", "YRInt2_G2", "YRNDSI", "YRNDVI", "YBS1", "YBS2", "YBS4", "YBS5_G1", 
"YBS5_G2", "YBSI", "YGS3", "YGSI1", "YGSI2", "YGSI3", "YGInt1", "YGInt2", "YNS6_G1", 
"YNS6_G2", "YNSI2_G1", "YNSI2_G2", "YNInt2_G1", "YNInt2_G2", "YNNDSI", "YNNDVI", 'pc1','pc2','pc3',"b1_1", 'b2_1', 'b3_1', 'b4_1','b8_asm'])



Run the next line. You can see the process of export in: https://code.earthengine.google.com/ or in https://code.earthengine.google.com/tasks

In [23]:
geemap.ee_export_image_to_drive(
    export, description='PS_Index_Cafine', folder='Planet_Images_Model_Salinity', region=geometry, scale= 3, crs = 'EPSG:2095'
)

#### 1.2.1 The image exported to Drive needs to be downloaded using PyDrive

In [11]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

credentials_path = 'C:/Users/cespe/client_secret_2_775448020170-glgpi5v37ur14of7spbms2gi67g7silt.apps.googleusercontent.com.json'

# Authenticate and create the PyDrive instance
gauth = GoogleAuth()
gauth.LoadClientConfigFile(credentials_path)
gauth.LocalWebserverAuth()  # Opens a browser window to authenticate
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=775448020170-glgpi5v37ur14of7spbms2gi67g7silt.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [24]:
# Google Drive folder ID
folder_id = '1iPqQkFH8ACsOgf0Pk8iZsiXpw0WqP6Sp'  # Replace with your actual folder ID
file_title = 'PS_Index_Cafine.tif'  # Name of the file you are looking for

try:
    # List all files in the specified folder
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
    
    # Iterate through the file list to find the file with the specified title
    for file in file_list:
        if file['title'] == file_title:
            # Download the file
            file.GetContentFile(file_title)
            print(f"File {file_title} downloaded successfully.")
            break
    else:
        # File not found in the folder
        print(f"File {file_title} not found in folder {folder_id}.")
except Exception as e:
    # Print any errors that occur during the process
    print(f"Error while searching for the file in Google Drive: {e}")



File PS_Index_Cafine.tif downloaded successfully.


#### 1.2.2 Extract the data of the image using the sample points and applying a buffer over the points

Add the sampling of Cafine to the enviroment 

In [26]:
import shapely as shp

# read in training data polygons that created as geojson from a shared directory
training_data = '../DataIn/Cafine.geojson'
training_vectors = gpd.read_file(training_data)

# make a bounding box and centroid for mapping
bbox = training_vectors.total_bounds
center = shp.geometry.box(bbox[0], bbox[1], bbox[2], bbox[3]).centroid

# show the 1st 5 lines
training_vectors.head()

,ogc_fid,id,ID_Sample,Sand,Clay,Silt,X,Y,Sample_ID,SAR,CE,geometry
0,1.0,1,M-CF 1,0.15,0.39,0.46,477148.9961,1240297.891,M-CF 1,26.926614,7.089333,POINT (477148.996 1240297.892)
1,2.0,2,M-CF 2,0.51,0.17,0.32,477148.9961,1240055.391,M-CF 2,86.327645,29.169333,POINT (477148.996 1240055.392)
2,3.0,3,M-CF 3,0.34,0.19,0.47,477148.9961,1239812.891,M-CF 3,112.754286,42.417333,POINT (477148.996 1239812.892)
3,4.0,4,M-CF 4,0.49,0.08,0.43,477148.9961,1239570.391,M-CF 4,193.130080,89.521333,POINT (477148.996 1239570.392)
4,5.0,5,M-CF 5,0.51,0.09,0.40,477148.9961,1239085.391,M-CF 5,126.060248,57.628000,POINT (477148.996 1239085.392)


In [27]:
# find all unique values of training data names to use as classes
classes = training_vectors[['CE']]
classes.shape

print(classes)

            CE
0     7.089333
1    29.169333
2    42.417333
3    89.521333
4    57.628000
..         ...
178  19.110667
179  39.718667
180  33.340000
181   3.900000
182  36.038667

[183 rows x 1 columns]


In [31]:
from shapely.geometry import shape, box
import rasterio
from rasterio.features import rasterize
import numpy as np
from rasterstats.io import bounds_window

# Define the buffer distance (in raster units or coordinates)
buffer_distance = 6  # Adjust this distance as needed

##If you want to read the data directly from the shared folder, uncomment the following line.
raster_file = '../DataIntermediate/PS_Index_Cafine.tif'

# Create an empty list to store the buffered geometries
buffer_geoms = []
X_raw = []

with rasterio.open(raster_file, 'r') as src:
    for (label, geom) in zip(training_vectors.SAR, training_vectors.geometry):
        # Create a buffered geometry
        buffered_geom = geom.buffer(buffer_distance)

        # Add the buffered geometry to the list
        buffer_geoms.append(buffered_geom)

        # Get the bounds of the buffered geometry
        buffered_bounds = buffered_geom.bounds

        # Use the buffered bounds to create the window
        window = bounds_window(buffered_bounds, src.transform)

        window_affine = src.window_transform(window)
        fsrc = src.read(window=window)
        mask = rasterize(
            [(buffered_geom, 1)],
            out_shape=fsrc.shape[1:],
            transform=window_affine,
            fill=0,
            dtype='float32',
            all_touched=True
        ).astype(bool)
        label_pixels = np.argwhere(mask)

        buffer_values = []  # Lista para almacenar los valores de píxeles en el buffer
        for (row, col) in label_pixels:
            data = fsrc[:, row, col]
            one_x = np.nan_to_num(data, nan=0)
            buffer_values.append(one_x)

        # Calcular el valor promedio dentro del buffer
        if buffer_values:
            average_value = np.median(buffer_values, axis=0)
            X_raw.append(average_value)

In [32]:
# Create a GeoDataFrame from the list of buffered geometries
gdf = gpd.GeoDataFrame(geometry=buffer_geoms)

# Define the output shapefile path on Google Drive
output_shapefile = '../DataIntermediate/buffers_cafine.shp'

# Save the GeoDataFrame as a shapefile
gdf.to_file(output_shapefile)

print("Archivos shapefile guardados con éxito en Google Drive.")

Archivos shapefile guardados con éxito en Google Drive.


In [33]:
import rasterio

# Input raster file path
raster_file = raster_file

# Open raster file in read-only mode
with rasterio.open(raster_file, 'r') as src:
    # print raster properties.
    print('Input raster properties:')
    print('---------------------------------')
    print('Number of bands:', src.count)
    print('Dimensions (rows, columns):', src.shape)
    print('Coordinate reference system (CRS):', src.crs)
    print('Data type:', src.dtypes[0]) # First band data type
    print('Nodata values:', src.nodata)


Input raster properties:
---------------------------------
Number of bands: 110
Dimensions (rows, columns): (1263, 2962)
Coordinate reference system (CRS): EPSG:2095
Data type: float32
Nodata values: None


In [35]:
import pandas as pd

coordinates = pd.DataFrame(training_vectors)
X = pd.DataFrame(X_raw, columns = ['b1','b2','b3','b4','b5','b6','b7','b8','NDVI','SR','GCVI','NDWI','VARI','GNDVI','GRVI','SAVI','VSSI','S1','S2','S3_G1','S3_G2','S4','S5_G1','S5_G2','S6_G1','S6_G2','SI','NDSI','SI1_G1','SI1_G2','SI2_G1','SI2_G2','SI3_G1','SI3_G2',
'Int1_G1','Int1_G2','Int2_G1','Int2_G2','YRS6_G1','YRS6_G2','RS6_G1','RS6_G2','RS1','RS2','RS3_G1','RS3_G2','RS4','RS5_G1','RS5_G2','RNDSI','RNDVI','RSI','RSI1_G1','YRSI1_G1', 'YRSI1_G2', 'YRSI2_G1',
'RSI1_G2','RSI2_G1','RSI2_G2','RSI3_G1','RSI3_G2','RInt1_G1','RInt1_G2','RInt2_G1','RInt2_G2', 'YRS1', 'YRS2', 'YRS3_G1', 'YRS3_G2', 'YRS4', 'YRS5_G1', 'YRS5_G2','YRSI',  'YRSI2_G2', 'YRSI3_G1',
'YRSI3_G2','YRInt1_G1', 'YRInt1_G2', 'YRInt2_G1', 'YRInt2_G2', 'YRNDSI', 'YRNDVI', 'YBS1','YBS2', 'YBS4', 'YBS5_G1', 'YBS5_G2', 'YBSI', 'YGS3', 'YGSI1', 'YGSI2', 'YGSI3','YGInt1', 'YGInt2', 'YNS6_G1',
'YNS6_G2', 'YNSI2_G1', 'YNSI2_G2', 'YNInt2_G1', 'YNInt2_G2','YNNDSI', 'YNNDVI','pc1','pc2','pc3','b1_1','b2_1','b3_1','b4_1','b8_asm']).dropna()
# X = pd.concat([X, coordinates[['x', 'y']]], axis=1)
y = pd.DataFrame(training_vectors[["CE",'id']]).dropna()

(X.shape, y.shape)

((183, 110), (183, 2))

In [36]:
# Concatenar X y y a lo largo de las columnas (axis=1)
merged_df_cafine = pd.concat([X, y], axis=1)

# Exportar el DataFrame fusionado a un archivo xlsx
merged_df_cafine.to_excel('../DataIntermediate/merged_data_cafine.xlsx', index=False)